In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import random

In [2]:
n=40428967
s_size=1000000
skip_vals=sorted(random.sample(range(1,n), n-s_size))

In [3]:
data=pd.read_csv(r'C:\Users\Dell\Downloads\train.csv', header=0, skiprows=skip_vals)

In [4]:
parse_date = lambda val : pd.datetime.strptime(val, '%y%m%d%H')
data['hour'] = pd.to_datetime(data['hour'],format = '%y%m%d%H') 

In [5]:
train_data=data.copy()
y_label=train_data.pop('click')
train_data=train_data.drop(['hour','id', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model'],axis=1)

In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(train_data,y_label,test_size = 0.2,random_state=21)

In [7]:
import lightgbm as lgb

##### the values of cdf where msk is True are assigned to the array train, and the values of cdf where msk is False are assigned to test. 80% of cdf to be partitioned into train, and the remaining ~20% to be partitioned into test.

In [9]:
msk = np.random.rand(len(x_train)) < 0.8

In [10]:
lgb_train = lgb.Dataset(x_train[msk], y_train[msk])
lgb_eval = lgb.Dataset(x_train[~msk], y_train[~msk], reference=lgb_train)

In [11]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': { 'binary_logloss'},
    'num_leaves': 31, # defauly leaves(31) amount for each tree
    'learning_rate': 0.08,
    'feature_fraction': 0.7, # will select 70% features before training each tree
    'bagging_fraction': 0.3, #feature_fraction, but this will random select part of data
    'bagging_freq': 5, #  perform bagging at every 5 iteration
    'verbose': 0
}

In [12]:
print('Start training...')
# train
gbm = lgb.train(params, lgb_train, num_boost_round=4000, valid_sets=lgb_eval, early_stopping_rounds=500)

Start training...
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.026598 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[1]	valid_0's binary_logloss: 0.451633
Training until validation scores don't improve for 500 rounds
[2]	valid_0's binary_logloss: 0.448035
[3]	valid_0's binary_logloss: 0.444787
[4]	valid_0's binary_logloss: 0.442096
[5]	valid_0's binary_logloss: 0.439891
[6]	valid_0's binary_logloss: 0.437837
[7]	valid_0's binary_logloss: 0.436041
[8]	valid_0's binary_logloss: 0.43455
[9]	valid_0's binary_logloss: 0.433211
[10]	valid_0's binary_logloss: 0.432053
[11]	valid_0's binary_logloss: 0.43098
[12]	valid_0's binary_logloss: 0.429943
[13]	valid_0's binary_logloss: 0.429116
[14]	valid_0's binary_logloss: 0.428314
[15]	valid_0's binary_logloss: 0.427604
[16]	valid_0's binary_logloss: 0.427025
[17]	valid_0's binary_logloss: 0.42641
[18]	valid_0's binary_l